In [1]:
import glob
import music21
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.utils import to_categorical

# Check if a GPU is available and set it as the default device
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU")

def get_notes():
    notes = []
    for file in glob.glob("D:/PARTH COLLEGE/CODEALPHA/midi_songs/*.mid"):
        midi = music21.converter.parse(file)
        notes_to_parse = None
        try:
            parts = music21.instrument.partitionByInstrument(midi)
            notes_to_parse = parts.parts[0].recurse()
        except:
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, music21.note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, music21.chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

def prepare_sequences(notes, pitchnames, n_vocab):
    sequence_length = 50  # Reduced sequence length
    network_input = []
    network_output = []
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    for i in range(0, len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
    network_output = to_categorical(network_output)
        
    return network_input, network_output

def create_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(
        256,  # Reduced number of units
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(256, return_sequences=True, recurrent_dropout=0.3))
    model.add(LSTM(256))
    model.add(Dense(128))  # Reduced number of units
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = network_input[start]
    prediction_output = []

    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        
        prediction = model.predict(prediction_input, verbose=0)
        
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]
    
    return prediction_output

def create_midi(prediction_output):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = music21.note.Note(int(current_note))
                new_note.storedInstrument = music21.instrument.Piano()
                notes.append(new_note)
            new_chord = music21.chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = music21.note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = music21.instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = music21.stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')

# Extract notes
notes = get_notes()

if notes:
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, pitchnames, n_vocab)
    
    model = create_network(network_input, n_vocab)

    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.keras"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=1,
        save_best_only=True,
        mode='min',
        save_weights_only=False,
        save_freq='epoch'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=2, batch_size=4192, callbacks=callbacks_list)  # Reduced epochs and increased batch size

    model.save('my_model.keras')

    # Generate notes after training
    generated_notes = generate_notes(model, network_input, pitchnames, n_vocab)

    # Create MIDI file
    create_midi(generated_notes)
else:
    print("No notes found.")


c:\Users\succe\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - loss: 5.2748
Epoch 1: loss improved from inf to 5.24345, saving model to weights-improvement-01-5.2434.keras
4/4 ━━━━━━━━━━━━━━━━━━━━ 31s 6s/step - loss: 5.2685
Epoch 2/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - loss: 4.9002
Epoch 2: loss improved from 5.24345 to 4.85866, saving model to weights-improvement-02-4.8587.keras
4/4 ━━━━━━━━━━━━━━━━━━━━ 26s 6s/step - loss: 4.8919
